In [ ]:
# Learning over Averaged-Pretrained-ICD9-for each stay to predict EndOfStay diagnosis
# Did not work - seq_lenght too short
# No relevant padding & unknown vectors found in pretrained embeddings - github.com/clinicalml/embeddings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/pau/Medical-Diagnosis-Learning/src')
from icd9_train import *

In [3]:
base_path = '/media/disk3/disk3'
patients, pat_adm_id = build_encounter_seq(os.path.join(base_path, 'mimic3/DIAGNOSES_ICD.csv'), False)

In [4]:
diff = []
for k, v in patients.iteritems():
    source_icds = [_[1] for i in v.keys()[:-1] for _ in v[i]]
    target_icds = [_[1] for _ in v[v.keys()[-1]]]
    diff.append(len(set(target_icds) - set(source_icds)))
print "Average Difference in # of ICDs in last encounter and previous ones:", sum(diff)/len(diff)

Average Difference in # of ICDs in last encounter and previous ones: 9


In [4]:
embeddings, token2idx, idx2token = read_icd9_embeddings(os.path.join(base_path, 'claims_codes_hs_300.txt'))

IOError: [Errno 2] No such file or directory: '/media/disk3/disk3/claims_codes_hs_300.txt'

In [6]:
labels=['4019','41401','42731','4280','25000','2724','5849','V053','51881','V290','2720','53081','5990','2859','2449','V3000','2851','486','2762','496']
db = ICD9DataBuilder(patients, labels, token2idx)
db.label_data()

In [20]:
import pickle
f = open(os.path.join(base_path, 'mimic_labelled_sample.pickle'), 'w')
pickle.dump({'data': db.data}, f)

In [12]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.embed_dim = embed_dim
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim)
        self.lin = nn.Linear(hidden_dim, len(labels))
    def init_hidden(self):
        return (Variable(torch.zeros(1, batch_size, hidden_dim)), Variable(torch.zeros(1, batch_size, hidden_dim)))
    def forward(self, x, hidden):
        x = self.embed(x)
        x = torch.transpose(x, 1, 0)
        x, _hidden  = self.lstm(x, hidden)
        x = x[-1, :, :].view(batch_size, -1)
        x = self.lin(x)
        return x
crit = nn.CrossEntropyLoss()

In [17]:
embedding_size = 300
hidden_dim = 100
batch_size=16
model = LSTMModel(len(token2idx.keys()), embedding_size, hidden_dim)
opti = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
step = 0
step_log = []
loss_log = []
val_acc_log = []

train_data = ICD9Dataset(db.data)
train_loader = torch.utils.data.DataLoader(dataset=train_data, shuffle=True,
                                batch_size=batch_size,collate_fn=padding_collation)

for batch in train_loader:
    model.zero_grad()
    batch_x, batch_y = batch[0], torch.LongTensor(batch[1])
    batch_emb = np.ndarray((batch_size, len(batch_x[0]), embedding_size))
    for i, enclist in enumerate(batch_x):
        for j, enc in enumerate(enclist):
            if enc == 0:
                batch_emb[i, j, :] = embeddings['<PAD>']
            else:
                batch_emb[i, j, :] = np.mean([embeddings[idx2token[icd]] for icd in enc], axis=0)    

    x = Variable(torch.from_numpy(batch_emb)).transpose(1,0)
    hidden = model.init_hidden()
    
    x = model(x, hidden)
    loss = crit(x, Variable(batch[1].view(-1)))
    loss.backward()
    opti.step()
    break
#     if step % 100 == 0:
#         val_acc = evaluate(model, val_loader)
#         print("Step: %d, Loss: %.4f, Validation Acc: %.2f"%(step, loss.data[0], val_acc))
#         step_log.append(step)
#         loss_log.append(loss.data[0])
#         val_acc.append(val_acc)
#     step += 1
    

AssertionError: 